In [ ]:
import json
import pandas as pd
import networkx as nx
import csv
from statistics import *
import networkx as nx

LIST_OF_FEATURES = ['avg_cl', 'trans', 'deg_centr', 'average_neighbor_degree', 'average_degree_connectivity', 'k_nearest_neighbors', 'degree_centrality',
 'closeness_centrality', 'betweenness_centrality', 'diameter', 'average_shortest_path_length']

vk_prof_df = pd.read_csv("VK_profiles_info.csv")
graph_final_file = open("graph_final.json")
graph = dict((int(k), v)for k, v in json.load(graph_final_file).items())
graph_final_file.close()

def make_graph_for_user(user_id_1, uid2friends):
    """Создание графа друзей Вконтакте пользователя с user_id."""

    graph = nx.Graph()
    graph.add_node(user_id_1)

    friends_ids = set(uid2friends[user_id_1])

    for friend_id in friends_ids:
        graph.add_edge(user_id_1, friend_id)
        friends_ids_2nd_gen = uid2friends[friend_id]
        for friend_id_2nd_gen in friends_ids_2nd_gen:
            if friend_id_2nd_gen in friends_ids:
                graph.add_edge(friend_id_2nd_gen, friend_id)
    return graph

def get_graph_features(graph_1, user_id): 
    avg_cl = nx.average_clustering(graph_1)
    trans = nx.transitivity(graph_1)
    deg_centr = mean(nx.degree_centrality(graph_1).values())
    average_neighbor_degree = mean(nx.average_neighbor_degree(graph_1))
    average_degree_connectivity = mean(nx.average_degree_connectivity(graph_1).values())
    k_nearest_neighbors = mean(nx.k_nearest_neighbors(graph_1).values())
    degree_centrality = mean(nx.degree_centrality(graph_1).values())
    closeness_centrality = mean(nx.closeness_centrality(graph_1).values())
    betweenness_centrality = mean(nx.betweenness_centrality(graph_1).values())
    diameter = nx.diameter(graph_1)
    radius = nx.radius(graph_1)
    average_shortest_path_length = nx.average_shortest_path_length(graph_1)
    features = [avg_cl, trans, deg_centr, average_neighbor_degree, average_degree_connectivity,k_nearest_neighbors,
           degree_centrality, closeness_centrality, betweenness_centrality,  diameter, radius, average_shortest_path_length]
    list_of_features = []
    for feature in features:
        if type(feature) == int or type(feature) == float:
            list_of_features.append(feature)
        else:
            pass
    return list_of_features

with open('Graph_data.csv', 'a') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=LIST_OF_FEATURES)
    writer.writeheader()
    for uid in vk_prof_df.uid:
        graph_1 = make_graph_for_user(uid, graph)
        feat = get_graph_features(graph_1, uid)
        i = 0
        a = {}
        for feature in LIST_OF_FEATURES:
            try:
                a[feature] = feat[i]
                i += 1
            except:
                pass
        writer.writerow(a)